# DASCore Training

August 14, 2025

This notebook shows an example DASCore application: visualizing and processing signals from signals generated by walking and hammer shots.

The walking data and sledge hammer shots were collected in Jan. 2025 and Jun. 2025, respectively, at the UNR farm test site.

<a target="_blank" href="https://colab.research.google.com/github/DASDAE/ctemps_tutorial/blob/master/03_application.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>  

#### Useful links: 
* [Colab link](https://colab.research.google.com/github/DASDAE/ctemps_tutorial/blob/master/03_application.ipynb)
* [DASCore tutorial](https://dascore.org/tutorial/concepts.html)
* [UNR farm test site map](https://drive.google.com/file/d/1v6teeDuYw9Mj33izdEsoNkBJ6ZU5I1eI/view?usp=drive_link)
  

In [ ]:
%%capture

# First ensure DASCore is installed. If not, install and restart the kernel.
try:
    import dascore as dc
except ImportError:
    !pip install dascore
    !pip install ipympl
    # resetart kernel
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

# need to stick to 3.9 for now as there is a dependency issue with higher versions that we are working on.
import matplotlib
if matplotlib.__version__ != "3.9.2":
  !pip install matplotlib==3.9.2 --quiet
  import IPython
  IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [ ]:
import gdown, zipfile
import numpy as np
from pathlib import Path


## Setup
First, we create a directory of DAS files to simulate the output of an acquisition. 

In [ ]:
# Paste the *public* Google Drive share URL 
url = "https://drive.google.com/file/d/15xMONKL4E00JANze20OW65xBYmPZGmDV/view?usp=sharing"
out = Path("ctemps_das_walking.zip")

# If the zip already exists, skip downloading
if not out.exists():
    # Ensure we save to the expected filename (avoid gdown renaming surprises)
    gdown.download(url=url, output=str(out), quiet=False, fuzzy=True)

das_dir = Path("ctemps_das_walking")

# If the das_dir already exists, skip unzipping
if not das_dir.exists():
    das_dir.mkdir(exist_ok=True)
    with zipfile.ZipFile(out, "r") as zf:
        zf.extractall(das_dir)
    print(f"Unzipped the downloaded data to: {das_dir.resolve()}")

## Exploration

We initialize a spool on the directory of DAS files and explore a summary of the contents.

In [ ]:
# Update will create an index of the contents for fast querying/access
spool = dc.spool(das_dir).update()

In [ ]:
spool.get_contents()

In [ ]:
patch = spool[0]
patch

There are 2 files, each 10 seconds of data. One example file looks like this:

In [ ]:
patch.viz.waterfall(scale=0.0001);

Quickly look at the data in f-k domain

In [ ]:
# Apply transform on all dimensions
fk_patch = patch.dft(patch.dims)

# We can't plot complex arrays so only plot amplitude
ax = fk_patch.abs().viz.waterfall(scale=0.1)

# Zoom in around interesting frequencies
ax.set_ylim(-500, 500);

### **Exercise** 
Use `Patch.select` to remove the channels outside of study area before performing discrete fourier transform to see if that helps to improve the f-k plot.

## Chunk

Let's merge the two patches and create a 20-second patch and select the channels we are interested in. Then, visualize the waterfall and spectrogram plots.

In [ ]:
sp_chunked = spool.chunk(time=None).select(distance=(50, 300))
merged_patch = sp_chunked[0].taper(distance=(0.1, 0.1))
merged_patch.viz.waterfall(scale=0.001);

The data is in rad/m/s. We can transform it to strain rate by applying a scaler as noted in OptoDAS interrogator's manual.


In [ ]:
# wave_length = ? # laser wave length (m)
# photoelastic = 0.78 # fiber photoelastic effect (as set in Settings, Acquistion tab)
# refractive = ? # refractive index (as set in the Measurement Settings)
# scaler = wave_length / (4 * np.pi * photoelastic * refractive)

# # Now, update the patch
# new_data_merged_patch = merged_patch.update(data=merged_patch.data * scaler)
# merged_patch = new_data_merged_patch.update_attrs(data_units="1 / s")

In [ ]:
ax = merged_patch.viz.spectrogram(scale=0.1)
ax.set_ylim(5000, 0);

## Low-pass filter and downsample
The data is recorded in 10 kHz. We can low-pass filter and down sample to better see low-frequency features.

In [ ]:
from dascore.units import Hz
cut_off = 500 # Hz
pa_lp = merged_patch.pass_filter(time=(None, cut_off*Hz)).taper(time=(0.1, 0.1))

dt = 1/(2*cut_off)
step = np.timedelta64(int(round(dt * 1e9)), "ns")
new_time_ax = np.arange(pa_lp.attrs["time_min"], pa_lp.attrs["time_max"], step)
pa_down_sampled = (
    pa_lp.interpolate(time=new_time_ax)
    .update_coords(time_step=dt)
)
pa_down_sampled.viz.waterfall(scale=(-2,2));

Let's compare the spectogram of data with original size and downsampled data.

In [ ]:
ax = pa_lp.viz.spectrogram(scale=0.01);
ax.set_ylim(1000, 0);

In [ ]:

ax = pa_down_sampled.viz.spectrogram(scale=0.1);
ax.set_ylim(500, 0);

Downsample the data even more.

In [ ]:
cut_off = 100
pa_bp = merged_patch.pass_filter(time=(None, cut_off*Hz)).taper(time=(0.1, 0.1))

dt = 1/(2*cut_off)
step = np.timedelta64(int(round(dt * 1e9)), "ns")
new_time_ax = np.arange(pa_bp.attrs["time_min"], pa_bp.attrs["time_max"], step)
pa_down_sampled = (
    pa_bp.interpolate(time=new_time_ax)
    .update_coords(time_step=dt)
)

In [ ]:
ax = pa_down_sampled.viz.spectrogram(scale=0.1);
ax.set_ylim(100, 0);

### Exercise 
Using the `Patch.pass_filter()` funtion, apply a band-pass filter of 1 to 30 Hz. Then, visualize the f-k plot.

Plot the f-k again for the downsampled data.

In [ ]:
fk_patch = pa_down_sampled.dft(pa_down_sampled.dims)

# We can't plot complex arrays so only plot amplitude
ax = fk_patch.abs().viz.waterfall(scale=0.1)

# Zoom in around interesting frequencies
ax.set_ylim(-100, 100);

## Slope filter

Now, after looking at the f-k plot and considering our signal of interest, we apply a slope filter to remove high-velocity signals.

In [ ]:
v1 = 0.1 # m/s
v2 = 20 # m/s
filt = np.array([0.1*v1, v1, v2, 10*v2])
patch_slope_filtered = pa_down_sampled.slope_filter(filt=filt)
patch_slope_filtered.viz.waterfall(scale=0.01)

### **Exercise** 
Use `Patch.rolling()` [function](https://dascore.org/api/dascore/proc/rolling/rolling.html) to improve denoising the data by applying a moving median over 0.1 second window with 0.5 step. Call it patch_filtered.
Finally, visualize and save the patch_filtered.

## Hammer shots

As a final challenge, let's work on the hammer shots data and apply different processing functions to enhance visualization. You can also use DASCore's [Patch.dispersion_phase_shift()](https://dascore.org/api/dascore/transform/dispersion/dispersion_phase_shift.html) funtion to perform dispersion analysis and characterize surface waves.

In [ ]:
# Paste the *public* Google Drive share URL 
url = "https://drive.google.com/file/d/1GrfUD5kLDC8Q1v4A7BUCrzalK1HA_HAI/view?usp=drive_link"
out = Path("ctemps_das_hammer_shots.zip")

# If the zip already exists, skip downloading
if not out.exists():
    # Ensure we save to the expected filename (avoid gdown renaming surprises)
    gdown.download(url=url, output=str(out), quiet=False, fuzzy=True)

das_dir = Path("ctemps_das_hammer_shots")

# If the das_dir already exists, skip unzipping
if not das_dir.exists():
    das_dir.mkdir(exist_ok=True)
    with zipfile.ZipFile(out, "r") as zf:
        zf.extractall(das_dir)
    print(f"Unzipped the downloaded data to: {das_dir.resolve()}")

In [ ]:
spool = dc.spool(das_dir).update()
spool

In [ ]:
patch = spool[0]